In [ ]:
!nvidia-smi

Sat May  6 13:48:01 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Installation

In [1]:
!pip install langchain
!pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 759.0/759.0 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 2.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 2.7 MB/s eta 0:00:00


In [2]:
import os
os.environ["OPENAI_API_KEY"]="..."

## LLMs: language model로부터 prediction들을 얻자.
가장 기본적인 LangChain의 building block은 input에 대해 LLM을 불러 내는 것이다. 해당 코드의 목적은 회사가 무슨 일을 하는가에 따라 회사명을 생성해 보는 것이다.

In [ ]:
from langchain.llms import OpenAI

# Random성을 가중시키기 위해 temperature를 높게 설정해본다.
llm = OpenAI(temperature=0.9)

## KO
text = "다채로운 양말을 만드는 회사에 어울리는 회사명은 무엇일까?"
print(llm(text))

## EN
text = "What would be a good company name for a company that makes colorful socks?"
print(llm(text))



★열정같이 따뜻해지는 양말★


Fuzzy Feet Toesies.


## Prompt Templates: LLMs를 위한 prompt들을 관리한다.
일반적으로 LLM이 어플리케이션에서 사용될 떄, 유저의 input을 그대로 LLM에 넣지는 않을 것이다.
대신에, 아마도 당신은 유저의 input을 받고 prompt를 구성한 다음 LLM에게 그것을 전달해 줄 것이다.

위 예시를 업데이트 해본다면, 우리는 유저에게 회사가 하는 일만을 입력 받고 그 정보들로 prompt의 format을 맞춰주는 것을 생각해 볼 수 있겠다. 이 작업은 LangChain을 통해서면 매우 쉽게 진행이 가능하다!

In [ ]:
from langchain.prompts import PromptTemplate

## KO
ko_prompt = PromptTemplate(
    input_variables=["product"],
    template="{product}를(을) 만드는 회사에 어울리는 좋은 이름이 뭘까?"
)
print(ko_prompt.format(product="다채로운 양말"))

## EN
prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?",
)

print(prompt.format(product="colorful socks"))

다채로운 양말를(을) 만드는 회사에 어울리는 좋은 이름이 뭘까?
What is a good name for a company that makes colorful socks?


## Chains: LLMs와 prompt들을 다중 스텝 워크플로우(multi-step workflow)에서 합쳐보자.
지금까지는 PromptTemplate와 LLM을 단독으로 사용했었다. 하지만 실제 어플리케이션에서는 하나씩 단독으로 사용되는것이 아나라 이들의 조합으로 활용될 것이다.

LangChain의 chain은 각각의 LLMs부터 또 다른 chain까지의 link들로 구성된다. chain의 가장 코어 타입은 LLMChain으로 PromptTemplate와 LLM으로 구성되어있다. 앞의 예시를 더욱 확징해 보면, user input을 받아 PromptTemplate을 통해 format시키고, 이를 LLM에 넣어주는 동작이 LLMChain 내부에서 일어날 것이다.

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain

llm = OpenAI(temperature=0.9)
prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?"
)

## EN
chain = LLMChain(llm=llm, prompt=prompt)

print(chain.run("colorfulk socks"))

## KO
ko_chain = LLMChain(llm=llm, prompt=ko_prompt)
print(ko_chain.run("다채로운 양말"))



Happy Socks & Co.


1. 컬러풀한 양말 스타일
2. 따뜻한 양말 특강
3. 양말 스트릭트
4. 다채로운 스타일 피트
5. 신뢰할 수 있는 양말 세트
6. 꾸며진 양말 전문가
7. 누구나 소중하게 어울리는 양말 클라우드
8. 지속적인 양말 디자인
9. 따뜻


## Agents: User의 input에 기반해 chain들을 동적으로 호출한다.
Agent는 LLM을 취해야 할 action이나 어떠한 순서로 동작할지를 정하는데 사용한다. action은 tool을 사용해 output을 관찰하거나 user에게 반환해주거나 둘 중 하나일 수 있다.

agents를 불러오기 위해 아래 concept들을 이해해 두어야 한다:


* Tool : 특정한 과제를 수행할 함수. 이는 뒤 예시와 같을 수 있다: Google Search, Database lookup, Python REPL, 다른 chain들. tool의 interface는 현재 input:str, output:str인 function이다. 
* LLM : agent를 지원하는 언어 모델.
* Agent : 사용될 agent. 이는 support agent class를 참조하는 문자열(str)이어야 한다. 이유는 해당 notebook(quickstart)는 간단하고, 높은 레벨의 API에 집중하고 있고, 이는 오직 기본적으로 지원되는 agent들만을 설명하고 있기 때문이다. custom agent를 만드는 doc은 추후 공개될 예정이다(현 23.05.08)



In [1]:
!pip install google-search-results

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32019 sha256=b26c61a8792db25c4ddc13c083639475e959386b3d867337ce3af0fef12ebc56
  Stored in directory: /root/.cache/pip/wheels/d3/b2/c3/03302d12bb44a2cdff3c9371f31b72c0c4e84b8d2285eeac53
Successfully built google-search-results


In [ ]:
import os
os.environ["SERPAPI_API_KEY"] = "..."

In [ ]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.llms import OpenAI

# 첫쨰, agent를 컨트롤하는데 사용될 language model을 불러온다.
llm = OpenAI(temperature=0)

# 그 다음, 사용할 몇가지 tool들을 불러온다. "llm-math" tool은 LLM을 사용하므로, llm도 같이 넣어주어야 한다.
tools = load_tools(["serpapi", "llm-math"], llm=llm)

# 마지막으로, agent를 tool, language model, agent의 type을 넣어 초기화한다.
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

# 이제 테스트를 해보자!
agent.run("What was the high temperature in SF yesterday in Fahrenheit? What is that number raised to the .023 power?")

## Memory: Chain과 Agent에 상태(State)를 더해주자.
chain과 agent가 이전 interaction에 대한 정보를 기억하는 "memory" concept를 가질 수 있게 한다. 가장 명확하고 간단한 예시는 chatvot을 디자인 할 때 인데, 챗봇의 더 좋은 발화능력을 위해 이전 대화를 저장해 둠으로써 맥락을 사용할 수 있게 만드는 것이다. 

LangChain은 위 목적에 부합하는 몇가지의 특정 chain들을 구현해 두었다. 이 notebook(quickstart)는 두 가지의 다른 타입의 memory에 대해 하나의 chain(ConversationChain)을 사용해 진행해 볼 것이다.
* ConversationChain은 기본적으로 이전 모든 input/output을 기억하고 전달할 context에 이를 모두 더하는 식의 매우 간단한 타입의 memory를 가지고 있다. 

In [7]:
from langchain import OpenAI, ConversationChain

llm = OpenAI(temperature=0)
conversation = ConversationChain(llm=llm, verbose=True)

## EN
output = conversation.predict(input="Hi there!")
print(output)

## EN
ko_conversation = ConversationChain(llm=llm, verbose=True)
ko_output = ko_conversation.predict(input="반가워!")
print(ko_output)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi there!
AI:

> Finished chain.
 Hi there! It's nice to meet you. My name is AI. What's your name?


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: 반가워!
AI:

> Finished chain.
 반가워! 나는 인공지능 챗봇이야. 나는 너에게 도움이 될 수 있을까?


In [8]:
## EN
output = conversation.predict(input="I'm doing well! Just having a conversation with an AI.")
print(output)

## KO
ko_output = ko_conversation.predict(input="잘 지내지! 어서 AI와 대화를 해보자.")
print(ko_output)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there!
AI:  Hi there! It's nice to meet you. My name is AI. What's your name?
Human: I'm doing well! Just having a conversation with an AI.
AI:

> Finished chain.
 That's great! It's always nice to have a conversation with someone new. What would you like to talk about?


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 반가워!
AI:  반가워! 나는 인공지능 챗봇이야. 나는 너에게 도움이 될 수 있을까?
Human: 잘 지내지! 어서 

## Language Model 어플리케이션을 만들기: Chat Models
LLM을 대신해서 chat model을 사용할 수 있다. Chat model들은 language model들의 변형체이다. Chat model들은 내부적으로 language model을 사용하지만, 이들이 노출하는 인터페이스는 약간 다르다: "text in, text out" API보다는, input output으로 "chat message"를 받는 interface를 노출한다. 

## Chat Model로부터 완성 Message 받기.
완성된 chat을 얻기 위해서는 chat model에게 하나 이상의 message들을 보내주어야 한다. 이떄 응답은 message일 것이다. 현재 LangChain에서 지원되는 message들의 type은 AIMessage, HumanMessage, SystemMessage, 그리고 ChatMessage이다 - ChatMessage는 임의의 parameter를 받는다. 대부분의 경우, HumanMessage, AIMessage, SystemMessage만을 다루게 될 것이다.

In [9]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

chat = ChatOpenAI(temperature=0)

In [11]:
chat([HumanMessage(content="Translate this sentence from English to Korean. I love programming.")])

AIMessage(content='저는 프로그래밍을 좋아합니다.', additional_kwargs={}, example=False)

In [12]:
# 여러개의 message들을 gpt-3.5-turbo또는 gpt-4 model에게 보낼 수 있다.
messages = [
    SystemMessage(content="You are a helpful assistant that translates English to Korean."),
    HumanMessage(content="I love programming.")
]

chat(messages)

AIMessage(content='저는 프로그래밍을 좋아합니다.', additional_kwargs={}, example=False)

한 단게 더 나아가 다중 message 세트들에 대해 generate 함수를 사용해 완성 message들을 생성할 수 있다. 이 결과는 LLMResult class일 것이며 추가적인 message parameter들이 담겨 있을 것이다.

In [13]:
batch_messages = [
    [
        SystemMessage(content="You are a helpful assistant that translates English to Korean."),
        HumanMessage(content="I love programming.")
    ],
    [
        SystemMessage(content="You are a helpful assistant that translates English to Korean."),
        HumanMessage(content="I love artificial intelligence.")
    ],
]
result = chat.generate(batch_messages)
result

LLMResult(generations=[[ChatGeneration(text='저는 프로그래밍을 좋아합니다.', generation_info=None, message=AIMessage(content='저는 프로그래밍을 좋아합니다.', additional_kwargs={}, example=False))], [ChatGeneration(text='저는 인공지능을 좋아합니다.', generation_info=None, message=AIMessage(content='저는 인공지능을 좋아합니다.', additional_kwargs={}, example=False))]], llm_output={'token_usage': {'prompt_tokens': 57, 'completion_tokens': 27, 'total_tokens': 84}, 'model_name': 'gpt-3.5-turbo'})